In [1]:
# %%capture
# !pip install unsloth
# Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# !pip install unsloth==2025.2.14 unsloth_zoo==2025.2.7

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/ETOHA

Mounted at /content/drive
/content/drive/MyDrive/ETOHA


In [4]:
behprepost_map = {
    0: "No",
    1: "Yes, pre-surgery only",
    2: "Yes, post-surgery only",
    3: "Yes, pre- and post-surgery",
    4: "Yes, no surgery"
}

diag_basis_map = {
    1: "Clinical examination only (history and physical exam)",
    2: "Clinical-diagnostic tests or exploratory surgery/autopsy (no microscopic confirmation)",
    4: "Specific biochemical and/or immunological laboratory tests",
    5: "Hematological or cytological confirmation (primary tumor or metastases); unclear if cytology or histology",
    6: "Histological confirmation of metastases only (including autopsy)",
    7: "Histological confirmation of the primary tumor (or unknown whether primary or metastases); possibly autopsy with histological confirmation"
}

diffgrad_map = {
    1: "Well differentiated (low-grade)",
    2: "Moderately differentiated (intermediate)",
    3: "Poorly differentiated (high-grade)",
    4: "Undifferentiated / anaplastic",
    9: "Unknown / not applicable / not determined"
}

behavior_map = {
    2: "In situ",
    3: "Malignant"
}

gender_map = {
    1: "Male",
    2: "Female"
}

her2_stat_map = {
    0: "0 (negative)",
    1: "1+ (negative)",
    2: "2+ (equivocal)",
    3: "3+ (positive)",
    4: "Not determined",
    7: "Not determined (7)",
    9: "Not assessable / unknown"
}

hr_stat_map = {
    0: "Negative",
    1: "Positive",
    9: "Not assessable / unknown"
}

mari_uitslag_map = {
    1: "MARI node negative",
    2: "ITC (≤ 0.2 mm)",
    3: "Micrometastasis (>0.2 - ≤2 mm)",
    4: "MARI node positive",
    5: "MARI node not removed",
    9: "Unknown outcome"
}

tumor_type_map = {
    501300: "Invasive breast carcinoma",
    502200: "Ductal carcinoma in situ",
    503200: "Lobular carcinoma in situ"
    # Add additional codes if you have them
}

neeja_map = {
    0: "No",
    1: "Yes"
}

neejaonb_map = {
    0: "No",
    1: "Yes",
    9: "Unknown"
}

swk_map = {
    0: "No",
    1: "Yes",
    8: "Not registered in this region"
}

vital_status_map = {
    0: "Alive",
    1: "Deceased"
}

later_map = {
    "1.0": "Left",
    "2.0": "Right",
    "X": "Unknown"
}

swk_uitslag_map = {
    1: "Negative sentinel node",
    2: "ITC (≤ 0.2 mm)",
    3: "Micrometastasis (>0.2 - ≤2 mm)",
    4: "Positive sentinel node (>2 mm)",
    9: "Sentinel node not found"
}

topo_sublok_map = {
    "C500": "Nipple/areola",
    "C501": "Central portion of breast",
    "C502": "Upper-inner quadrant",
    "C503": "Lower-inner quadrant",
    "C504": "Upper-outer quadrant",
    "C505": "Lower-outer quadrant",
    "C506": "Axillary extension (tail)",
    "C508": "Overlapping area of breast",
    "C509": "Breast, NOS"
}

morphology_map = {
    8000: "Neoplasm, NOS",
    8001: "Malignant tumor cells",
    8004: "Malignant tumor, spindle cell type",
    8010: "Carcinoma, NOS",
    8012: "Large cell carcinoma, NOS",
    8013: "Large cell neuroendocrine carcinoma",
    8020: "Undifferentiated carcinoma, NOS",
    8022: "Pleomorphic carcinoma",
    8030: "Giant cell and spindle cell carcinoma",
    8032: "Spindle cell carcinoma, NOS",
    8033: "Pseudosarcomatous carcinoma",
    8035: "Carcinoma with osteoclast-like giant cells",
    8041: "Small cell carcinoma, NOS",
    8045: "Mixed small and large cell carcinoma",
    8046: "Non-small cell carcinoma",
    8070: "Squamous cell carcinoma, NOS",
    8071: "Keratinizing squamous cell carcinoma",
    8074: "Spindle cell squamous carcinoma",
    8082: "Lymphoepithelial carcinoma",
    8140: "Adenocarcinoma, NOS",
    8141: "Scirrhous adenocarcinoma",
    8145: "Diffuse type adenocarcinoma",
    8200: "Adenoid cystic carcinoma",
    8201: "Cribriform carcinoma",
    8211: "Tubular adenocarcinoma",
    8230: "Solid carcinoma, NOS",
    8240: "Neuroendocrine tumor, NOS / Grade 1 (carcinoid)",
    8244: "Mixed adenoneuroendocrine carcinoma (MANEC)",
    8246: "Neuroendocrine carcinoma, NOS",
    8249: "Neuroendocrine tumor, grade 2/3 (atypical carcinoid)",
    8255: "Adenocarcinoma with mixed subtypes",
    8260: "Papillary adenocarcinoma, NOS",
    8290: "Oncocytic adenoma / carcinoma (Hurthle cell carcinoma)",
    8310: "Clear cell adenocarcinoma, NOS",
    8314: "Lipid-rich carcinoma",
    8315: "Glycogen-rich carcinoma",
    8401: "Apocrine adenocarcinoma",
    8407: "Microcystic adnexal carcinoma / Sclerosing sweat gland carcinoma",
    8410: "Sebaceous gland adenocarcinoma",
    8430: "Mucoepidermoid carcinoma",
    8441: "Serous cystadenocarcinoma, NOS",
    8470: "Mucinous cystadenocarcinoma, NOS",
    8480: "Mucinous adenocarcinoma",
    8481: "Mucin-producing adenocarcinoma",
    8490: "Signet ring cell carcinoma / 'poorly cohesive' carcinoma",
    8500: "Ductal carcinoma, NOS",
    8501: "Comedocarcinoma, NOS",
    8502: "Secretory carcinoma",
    8503: "Intraductal papillary adenocarcinoma",
    8504: "Encapsulated (intracystic) papillary carcinoma",
    8507: "Intraductal micropapillary carcinoma",
    8508: "Cystic hypersecretory carcinoma",
    8509: "Solid papillary carcinoma",
    8510: "Medullary carcinoma, NOS",
    8512: "Medullary carcinoma with lymphoid stroma",
    8513: "Atypical medullary carcinoma",
    8514: "Ductal carcinoma, desmoplastic type",
    8519: "Pleomorphic lobular carcinoma in situ",
    8520: "Lobular carcinoma, NOS",
    8521: "Ductular carcinoma",
    8522: "Ductal and lobular carcinoma",
    8523: "Ductal carcinoma mixed with another carcinoma type",
    8524: "Lobular carcinoma mixed with another carcinoma type",
    8530: "Inflammatory carcinoma",
    8540: "Paget's disease of the breast",
    8541: "Paget's disease + invasive ductal carcinoma",
    8543: "Paget's disease + intraductal carcinoma (DCIS)",
    8550: "Acinar cell carcinoma",
    8560: "Adenosquamous carcinoma",
    8562: "Epithelial-myoepithelial carcinoma",
    8570: "Adenocarcinoma with squamous metaplasia",
    8571: "Adenocarcinoma with cartilaginous or bony metaplasia",
    8572: "Adenocarcinoma with spindle cell metaplasia",
    8573: "Adenocarcinoma with apocrine metaplasia",
    8574: "Adenocarcinoma with neuroendocrine differentiation",
    8575: "Metaplastic carcinoma, NOS",
    8980: "Carcinosarcoma, NOS",
    8982: "Myoepithelial carcinoma",
    8983: "Malignant adenomyoepithelioma"
}

therapie_map = {
    100000: "Surgery, NOS",
    120000: "Local tumor resection",
    "130C50": "Breast-conserving surgery, NOS",
    "131C50": "Lumpectomy (without axillary lymph node dissection)",
    "132C50": "Lumpectomy (with axillary lymph node dissection)",
    "140C50": "Non-breast-conserving surgery, NOS",
    "141C50": "Mastectomy (without axillary lymph node dissection)",
    "142C50": "Mastectomy (with axillary lymph node dissection)",
    190000: "Resection for another indication (incidental finding)",
    315000: "Regional lymph node dissection for metastases",
    690100: "Surgical treatment abroad"
}

FEATURE_PROMPT_LABELS = {
    "leeft": "Age",
    "gesl": "Gender",
    "incjr": "IncidentYear",
    "tumsoort": "TumorType",
    "diag_basis": "DiagnosisBasis",
    "topo_sublok": "TopographySub",
    "later": "Laterality",
    "morf": "Morphology",
    "gedrag": "Behavior",
    "diffgrad": "DifferentiationGrade",
    "stadium": "Stage",
    "ond_lymf": "NumLymphExamined",
    "pos_lymf": "NumLymphPositive",
    "er_stat": "ERstatus",
    "pr_stat": "PRstatus",
    "her2_stat": "HER2status",
    "dcis_comp": "DCIScomponent",
    "multifoc": "Multifocality",
    "tum_afm": "TumorSize(mm)",
    "swk": "SentinelProcedure",
    "swk_uitslag": "SentinelOutcome",
    "okd": "AxillaryDissection",
    "org_chir": "OrganSurgery",
    "uitgebr_chir_code": "SurgeryCode",
    "dir_reconstr": "DirectReconstruction",
    "chemo": "Chemo",
    "target": "TargetTherapy",
    "horm": "HormoneTherapy",
    "rt": "Radiotherapy",
    "meta_rt": "RTMetastases",
    "meta_chir": "SurgeryMetastases"
}
FEATURES_TO_INCLUDE = list(FEATURE_PROMPT_LABELS.keys())

In [ ]:
!pip install datasets

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen2.5-7B-Instruct_2_epochs_new_prompt_fold_0", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

In [6]:
from datasets import load_from_disk
from datasets import Dataset

base_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN


from datasets import load_from_disk
from datasets import Dataset

base_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN


def build_prompt(example):
    """
    Construct a descriptive string prompt from the row's data,
    mapping coded values to human-readable text.
    """

    gender_str = gender_map.get(example["gesl"], f"Unknown(gender={example['gesl']})")
    tumor_type_str = tumor_type_map.get(example["tumsoort"], f"Unknown(tumsoort={example['tumsoort']})")
    behavior_str = behavior_map.get(example["gedrag"], f"Unknown(gedrag={example['gedrag']})")
    diff_str = diffgrad_map.get(example["diffgrad"], f"Unknown(diff={example['diffgrad']})")
    later_key = str(example["later"])
    later_str = later_map.get(later_key, f"Unknown(later={example['later']})")
    topo_sub_str = topo_sublok_map.get(example["topo_sublok"], f"Unknown(topo_sublok={example['topo_sublok']})")
    morph_str = morphology_map.get(example["morf"], f"Unknown(morf={example['morf']})")
    er_str = hr_stat_map.get(example["er_stat"], f"Unknown(ER={example['er_stat']})")
    pr_str = hr_stat_map.get(example["pr_stat"], f"Unknown(PR={example['pr_stat']})")
    her2_str = her2_stat_map.get(example["her2_stat"], f"Unknown(HER2={example['her2_stat']})")

    # 🔹 Handle cases where survival status may be missing (in inference)
    if "os" in example:
        status_str = "Alive" if example["os"] == 0 else "Deceased"
        status_line = f"Patient status: {status_str}."
    else:
        status_line = "Patient status: Unknown (predict survival time based on patient data)."

    prompt_lines = [
        f"Patient age: {int(example['leeft'])} years old.",
        f"Gender: {gender_str}.",
        f"Incident year: {int(example['incjr'])}.",
        f"Tumor type: {tumor_type_str}, morphology: {morph_str}, behavior: {behavior_str}.",
        f"Location: {later_str} breast, sublocation: {topo_sub_str}.",
        f"Differentiation grade: {diff_str}.",
        f"ER status: {er_str}, PR status: {pr_str}, HER2 status: {her2_str}.",
        status_line  # Adaptively includes status or a guiding note if missing
    ]

    # 🔹 Instruction explicitly tells the model what to do in all scenarios
    instruction = (
        "Predict the number of days the patient has survived or is expected to survive based on the provided medical data.\n\n"
        "- If the patient is marked as 'Deceased', predict the actual survival duration in days.\n"
        "- If the patient is marked as 'Alive', estimate the expected survival time based on similar cases.\n"
        "- If the patient status is not provided, use all available data to infer the most probable survival duration."
    )

    input_text = "\n".join(prompt_lines)
    response = str(example["time_to_os"])

    return {"instruction": instruction, "input": input_text, "response": response, "os": example["os"]}


def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["response"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = base_prompt.format(instruction, input_text, "") #+ EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# **Load Dataset**
raw_dataset = load_from_disk("./datasets/testfold_0")

new_dataset = []
for sample in raw_dataset:
    row = build_prompt(sample)
    new_dataset.append(row)

dataset = Dataset.from_list(new_dataset)
print(dataset)

dataset = dataset.map(formatting_prompts_func, batched=True, )
print(dataset[0])

Dataset({
    features: ['instruction', 'input', 'response', 'os'],
    num_rows: 12000
})


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

{'instruction': "Predict the number of days the patient has survived or is expected to survive based on the provided medical data.\n\n- If the patient is marked as 'Deceased', predict the actual survival duration in days.\n- If the patient is marked as 'Alive', estimate the expected survival time based on similar cases.\n- If the patient status is not provided, use all available data to infer the most probable survival duration.", 'input': 'Patient age: 93 years old.\nGender: Female.\nIncident year: 2013.\nTumor type: Invasive breast carcinoma, morphology: Ductal carcinoma, NOS, behavior: Malignant.\nLocation: Left breast, sublocation: Overlapping area of breast.\nDifferentiation grade: Moderately differentiated (intermediate).\nER status: Positive, PR status: Negative, HER2 status: 3+ (positive).\nPatient status: Deceased.', 'response': '390', 'os': 1, 'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appr

In [ ]:
# import json
# from tqdm import tqdm

# result = {"real": [], "predicted": [], "os": [], "total_predictions": []}

# for sample in tqdm(dataset):
#     result["real"].append(sample['response'])
#     result["os"].append(sample["os"])
#     inputs = tokenizer(
#     [
#         sample["text"]

#     ], return_tensors = "pt").to("cuda")

#     total_predictions = []
#     for i in range(5):
#         outputs = model.generate(**inputs, max_new_tokens = 10, use_cache = True)
#         response = tokenizer.batch_decode(outputs)
#         response = response[0].split("### Response:\n")[1].replace("<|end_of_text|>", "")
#         if response.isnumeric():
#             total_predictions.append(int(response))
#     result["total_predictions"].append(total_predictions)
#     mean_response = sum(total_predictions)/len(total_predictions) if len(total_predictions) > 0 else 0
#     result["predicted"].append(mean_response)
#     if len(result["predicted"]) % 50 == 0:
#         with open("qwen_results.json", "w") as f:
#             json.dump(result, f)

# with open("qwen_results.json", "w") as f:
#     json.dump(result, f)

In [8]:
import json
from tqdm import tqdm

result = {"real": [], "predicted": [], "os": []}

for sample in tqdm(dataset):
    result["real"].append(sample['response'])
    result["os"].append(sample["os"])
    inputs = tokenizer(
    [
        sample["text"]

    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 10, use_cache = True)
    response = tokenizer.batch_decode(outputs)
    response = response[0].split("### Response:\n")[1].replace("<|im_end|>", "")

    result["predicted"].append(response)
    if len(result["predicted"]) % 50 == 0:
        with open("qwen_results_fold_0.json", "w") as f:
            json.dump(result, f)

with open("qwen_results_fold_0.json", "w") as f:
    json.dump(result, f)

100%|██████████| 12000/12000 [1:54:41<00:00,  1.74it/s]
